<a href="https://colab.research.google.com/github/eileenCHEN-9/2024e/blob/main/Creating_spatial_lags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install libpysal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 8.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import geopandas as gpd
from libpysal import graph
import os
from libpysal.weights import Rook, Queen, W
import matplotlib.pyplot as plt
import libpysal
import libpysal.weights

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/eileenCHEN-9/2024e/main/data/Data_clubs_04_interpolated.csv", encoding = "utf-8")
df.columns

Index(['id', 'city', 'year', 'trend_lg_agri_predicted',
       'trend_lg_nonagri_predicted', 'trend_lg_gdppc_predicted', 'v1', 'v8',
       'province_id', 'province', 'lg_nonagri_predicted', 'lg_agri_predicted',
       'lg_gdppc_predicted', 'lg_totalgdp_predicted', 'lg101214',
       'urban_population', 'total_population', 'emp_agri', 'emp_ind',
       'emp_ser', 'emp_agri_gr', 'emp_nonagri', 'emp_nonagri_gr', 'agri_gdp',
       'nonagri_gdp', 'non_agri_share', 'agri_gdp_empgr', 'nonagri_gdp_empgr',
       'ln_total_population', 'urban_pop_percent',
       'club_trend_lg_gdppc_predicted', 'club_trend_lg_nonagri_predicted',
       'club_trend_lg_agri_predicted', 'fclub_trend_lg_gdppc_predicted',
       'fclub_trend_lg_nonagri_predicted', 'fclub_trend_lg_agri_predicted',
       're_trend_lg_gdppc_predicted', 're_trend_lg_nonagri_predicted',
       're_trend_lg_agri_predicted', 'empnonagri_interpolated',
       'empagri_interpolated', 'empnonagri_gr_new', 'empagri_gr_new', 'lg_pop',
     

In [ ]:
# Generate unique IDs for each region
unique_regions = df['city'].unique()
region_to_id = {city: i for i, city in enumerate(unique_regions)}

# Create a new column with region IDs
df['region_id'] = df['city'].map(region_to_id)
df = df.set_index(["region_id","year"])
df

id city  trend_lg_agri_predicted  \
region_id year                                         
0         2004  110000  北京市                 4.318491   
          2005  110000  北京市                 4.384820   
          2006  110000  北京市                 4.451082   
          2007  110000  北京市                 4.517107   
          2008  110000  北京市                 4.582562   
...                ...  ...                      ...   
366       2016  659011  新星市                 4.952979   
          2017  659011  新星市                 4.999036   
          2018  659011  新星市                 5.044239   
          2019  659011  新星市                 5.089039   
          2020  659011  新星市                 5.133708   

                trend_lg_nonagri_predicted  trend_lg_gdppc_predicted    v1  \
region_id year                                                               
0         2004                    9.022714                 10.793742     3   
          2005                    9.116159                 10.876256     4   
          2006                    9.209022                 10.958215     5   
          2007                    9.300501                 11.038864     6   
          2008                    9.389754                 11.117413     7   
...                                    ...                       ...   ...   
366       2016                    8.761851                 10.716098  7335   
          2017                    8.860108                 10.791734  7336   
          2018                    8.955623                 10.865422  7337   
          2019                    9.049568                 10.938036  7338   
          2020                    9.142929                 11.010272  7339   

                  v8  province_id province  lg_nonagri_predicted  ...  \
region_id year                                                    ...   
0         2004  1415       110000  Beijing              8.789451  ...   
          2005  1786       110000  Beijing              9.029587  ...   
          2006  2157       110000  Beijing              9.191402  ...   
          2007  2528       110000  Beijing              9.297535  ...   
          2008  2899       110000  Beijing              9.425364  ...   
...              ...          ...      ...                   ...  ...   
366       2016  5933       650000      NaN              8.800174  ...   
          2017  6304       650000      NaN              8.821374  ...   
          2018  6675       650000      NaN              8.880743  ...   
          2019  7046       650000      NaN              8.889171  ...   
          2020  7417       650000      NaN              8.909250  ...   

                re_trend_lg_gdppc_predicted  re_trend_lg_nonagri_predicted  \
region_id year                                                               
0         2004                     1.131587                       1.511986   
          2005                     1.128593                       1.499803   
          2006                     1.125637                       1.488041   
          2007                     1.122703                       1.476686   
          2008                     1.119784                       1.465743   
...                                     ...                            ...   
366       2016                     1.013177                       1.227258   
          2017                     1.013955                       1.228100   
          2018                     1.014679                       1.228847   
          2019                     1.015372                       1.229543   
          2020                     1.016051                       1.230217   

                re_trend_lg_agri_predicted  empnonagri_interpolated  \
region_id year                                                        
0         2004                    1.063446                92.800003   
          2005                    1.062544                      NaN   
          2006             

In [ ]:
import urllib.request
url = "https://raw.githubusercontent.com/eileenCHEN-9/2024e/main/data/city_edited_thiessen.gal"
response = urllib.request.urlopen(url)
data = response.read()
text = data.decode('utf-8')

In [ ]:
with open('city_edited_thiessen.gal', 'w') as f:
  f.write(text)

w = libpysal.io.open("city_edited_thiessen.gal").read()

In [ ]:
w.transform = 'r'
pd.DataFrame(w)

,0,1
0,340800,"{'341700': 0.125, '340100': 0.125, '341500': 0..."
1,340300,"{'341600': 0.16666666666666666, '341100': 0.16..."
2,341600,"{'340300': 0.16666666666666666, '341200': 0.16..."
3,341700,"{'340800': 0.25, '341000': 0.25, '340700': 0.2..."
4,341100,"{'340300': 0.125, '340100': 0.125, '340400': 0..."
...,...,...
362,330600,"{'330500': 0.16666666666666666, '330400': 0.16..."
363,331000,"{'330700': 0.2, '330200': 0.2, '330600': 0.2, ..."
364,330300,"{'350100': 0.16666666666666666, '350900': 0.16..."
365,330900,"{'220600': 0.2, '371000': 0.2, '310000': 0.2, ..."


In [ ]:
df.columns

Index(['id', 'city', 'trend_lg_agri_predicted', 'trend_lg_nonagri_predicted',
       'trend_lg_gdppc_predicted', 'v1', 'v8', 'province_id', 'province',
       'lg_nonagri_predicted', 'lg_agri_predicted', 'lg_gdppc_predicted',
       'lg_totalgdp_predicted', 'lg101214', 'urban_population',
       'total_population', 'emp_agri', 'emp_ind', 'emp_ser', 'emp_agri_gr',
       'emp_nonagri', 'emp_nonagri_gr', 'agri_gdp', 'nonagri_gdp',
       'non_agri_share', 'agri_gdp_empgr', 'nonagri_gdp_empgr',
       'ln_total_population', 'urban_pop_percent',
       'club_trend_lg_gdppc_predicted', 'club_trend_lg_nonagri_predicted',
       'club_trend_lg_agri_predicted', 'fclub_trend_lg_gdppc_predicted',
       'fclub_trend_lg_nonagri_predicted', 'fclub_trend_lg_agri_predicted',
       're_trend_lg_gdppc_predicted', 're_trend_lg_nonagri_predicted',
       're_trend_lg_agri_predicted', 'empnonagri_interpolated',
       'empagri_interpolated', 'empnonagri_gr_new', 'empagri_gr_new', 'lg_pop',
       'agri_

In [ ]:
df2 = df[['lg_nonagri_predicted', 'lg_agri_predicted', 'lg_gdppc_predicted', 'lg_totalgdp_predicted', 'lg101214', 'urban_population',
       'total_population', 'emp_agri', 'emp_ind', 'emp_ser', 'emp_agri_gr',
       'emp_nonagri', 'emp_nonagri_gr', 'agri_gdp', 'nonagri_gdp',
       'non_agri_share', 'agri_gdp_empgr', 'nonagri_gdp_empgr',
       'ln_total_population', 'urban_pop_percent', 'empagri_interpolated', 'empnonagri_gr_new', 'empagri_gr_new', 'lg_pop',
       'agri_inter_new', 'nonagri_inter_new']]
df2

lg_nonagri_predicted  lg_agri_predicted  lg_gdppc_predicted  \
region_id year                                                                
0         2004              8.789451           4.291225           10.571342   
          2005              9.029587           4.344534           10.797304   
          2006              9.191402           4.386100           10.943153   
          2007              9.297535           4.452595           11.035969   
          2008              9.425364           4.576198           11.153507   
...                              ...                ...                 ...   
366       2016              8.800174           4.973866           10.743710   
          2017              8.821374           4.931837           10.746368   
          2018              8.880743           4.971523           10.794847   
          2019              8.889171           5.032710           10.811186   
          2020              8.909250           5.081491           10.858846   

                lg_totalgdp_predicted  lg101214  urban_population  \
region_id year                                                      
0         2004               8.800518  9.035707      1.157983e+07   
          2005               9.038776  9.028996      1.165522e+07   
          2006               9.199555  9.029774      1.177870e+07   
          2007               9.305372  9.015269      1.190785e+07   
          2008               9.433168  9.002544      1.200395e+07   
...                               ...       ...               ...   
366       2016               8.821730  6.633253      4.704604e+03   
          2017               8.841622  6.640859      4.570604e+03   
          2018               8.900600  6.653890      4.580863e+03   
          2019               8.910093  6.661962      4.582604e+03   
          2020               8.930776  6.640736      4.596604e+03   

                total_population  emp_agri    emp_ind    emp_ser  ...  \
region_id year                                                    ...   
0         2004      1.417050e+07  7.199997  27.260000  65.540001  ...   
          2005      1.425188e+07  7.089996  26.320000  66.589996  ...   
          2006      1.433546e+07  0.519997  27.639999  71.839996  ...   
          2007      1.442160e+07  0.500000  26.459999  73.040001  ...   
          2008      1.451120e+07  0.459999  24.590000  74.949997  ...   
...                          ...       ...        ...        ...  ...   
366       2016      1.683713e+04       NaN        NaN        NaN  ...   
          2017      1.685146e+04       NaN        NaN        NaN  ...   
          2018      1.694032e+04       NaN        NaN        NaN  ...   
          2019      1.699305e+04       NaN        NaN        NaN  ...   
          2020      1.704556e+04       NaN        NaN        NaN  ...   

                agri_gdp_empgr  nonagri_gdp_empgr  ln_total_population  \
region_id year                                                           
0         2004     -125.656151       11291.155273            16.466673   
          2005       -8.476176         918.105103            16.472399   
          2006     -527.745239       64467.484375            16.478247   
          2007       -1.716988         218.221786            16.484238   
          2008       -3.885774         495.956696            16.490431   
...                        ...                ...                  ...   
366       2016             NaN                NaN             9.731341   
          2017             NaN                NaN             9.732192   
          2018             NaN                NaN             9.737452   
          2019             NaN                NaN             9.740560   
          2020             NaN                NaN             9.743645   

                urban_pop_percent  empagri_interpolated  empnonagri_gr_new  \
region_id year                                                               
0         2004       

In [ ]:
df2.columns

Index(['lg_nonagri_predicted', 'lg_agri_predicted', 'lg_gdppc_predicted',
       'lg_totalgdp_predicted', 'lg101214', 'urban_population',
       'total_population', 'emp_agri', 'emp_ind', 'emp_ser', 'emp_agri_gr',
       'emp_nonagri', 'emp_nonagri_gr', 'agri_gdp', 'nonagri_gdp',
       'non_agri_share', 'agri_gdp_empgr', 'nonagri_gdp_empgr',
       'ln_total_population', 'urban_pop_percent', 'empagri_interpolated',
       'empnonagri_gr_new', 'empagri_gr_new', 'lg_pop', 'agri_inter_new',
       'nonagri_inter_new'],
      dtype='object')

In [ ]:
lags = pd.DataFrame(index=df2.index, columns=['W_'+ col for col in df2.columns])

for year in df2.index.get_level_values('year').unique():
    for var in df2.columns:
        vals = df2.loc[pd.IndexSlice[:, year], var]
        lags.loc[vals.index, 'W_'+var] = libpysal.weights.lag_spatial(w, vals)
print(lags)

               W_lg_nonagri_predicted W_lg_agri_predicted  \
region_id year                                              
0         2004               6.770653            4.066455   
          2005               6.931561            4.112406   
          2006               7.117061            4.161437   
          2007               7.219121            4.247332   
          2008               7.346077            4.342672   
...                               ...                 ...   
366       2016               7.217662            4.956714   
          2017               7.295691            4.919156   
          2018               7.357481            4.950441   
          2019               7.366583            5.009233   
          2020               7.350762            5.075726   

               W_lg_gdppc_predicted W_lg_totalgdp_predicted W_lg101214  \
region_id year                                                           
0         2004             9.905166                6.89412

In [ ]:
lags.reset_index(inplace=True)
df.reset_index(inplace=True)

In [ ]:
df.columns

Index(['region_id', 'year', 'id', 'city', 'trend_lg_agri_predicted',
       'trend_lg_nonagri_predicted', 'trend_lg_gdppc_predicted', 'v1', 'v8',
       'province_id', 'province', 'lg_nonagri_predicted', 'lg_agri_predicted',
       'lg_gdppc_predicted', 'lg_totalgdp_predicted', 'lg101214',
       'urban_population', 'total_population', 'emp_agri', 'emp_ind',
       'emp_ser', 'emp_agri_gr', 'emp_nonagri', 'emp_nonagri_gr', 'agri_gdp',
       'nonagri_gdp', 'non_agri_share', 'agri_gdp_empgr', 'nonagri_gdp_empgr',
       'ln_total_population', 'urban_pop_percent',
       'club_trend_lg_gdppc_predicted', 'club_trend_lg_nonagri_predicted',
       'club_trend_lg_agri_predicted', 'fclub_trend_lg_gdppc_predicted',
       'fclub_trend_lg_nonagri_predicted', 'fclub_trend_lg_agri_predicted',
       're_trend_lg_gdppc_predicted', 're_trend_lg_nonagri_predicted',
       're_trend_lg_agri_predicted', 'empnonagri_interpolated',
       'empagri_interpolated', 'empnonagri_gr_new', 'empagri_gr_new', 'l

In [ ]:
lags.columns

Index(['region_id', 'year', 'W_lg_nonagri_predicted', 'W_lg_agri_predicted',
       'W_lg_gdppc_predicted', 'W_lg_totalgdp_predicted', 'W_lg101214',
       'W_urban_population', 'W_total_population', 'W_emp_agri', 'W_emp_ind',
       'W_emp_ser', 'W_emp_agri_gr', 'W_emp_nonagri', 'W_emp_nonagri_gr',
       'W_agri_gdp', 'W_nonagri_gdp', 'W_non_agri_share', 'W_agri_gdp_empgr',
       'W_nonagri_gdp_empgr', 'W_ln_total_population', 'W_urban_pop_percent',
       'W_empagri_interpolated', 'W_empnonagri_gr_new', 'W_empagri_gr_new',
       'W_lg_pop', 'W_agri_inter_new', 'W_nonagri_inter_new'],
      dtype='object')

In [ ]:
merged_df = df.merge(lags, on = ["region_id", "year"], how = "left")
merged_df

,region_id,year,id,city,trend_lg_agri_predicted,trend_lg_nonagri_predicted,trend_lg_gdppc_predicted,v1,v8,province_id,...,W_agri_gdp_empgr,W_nonagri_gdp_empgr,W_ln_total_population,W_urban_pop_percent,W_empagri_interpolated,W_empnonagri_gr_new,W_empagri_gr_new,W_lg_pop,W_agri_inter_new,W_nonagri_inter_new
0,0,2004,110000,北京市,4.318491,9.022714,10.793742,3,1415,110000,...,-21.619655,296.961144,15.372538,0.429309,1.86375,0.237499,-0.237499,15.372538,-22.271645,296.958027
1,0,2005,110000,北京市,4.384820,9.116159,10.876256,4,1786,110000,...,-41.871937,324.588127,15.377363,0.431172,NaN,NaN,NaN,15.377363,NaN,NaN
2,0,2006,110000,北京市,4.451082,9.209022,10.958215,5,2157,110000,...,-1.906325,185.828658,15.384064,0.445648,NaN,NaN,NaN,15.384064,NaN,NaN
3,0,2007,110000,北京市,4.517107,9.300501,11.038864,6,2528,110000,...,-4.939359,43.425798,15.389886,0.451625,NaN,NaN,NaN,15.389886,NaN,NaN
4,0,2008,110000,北京市,4.582562,9.389754,11.117413,7,2899,110000,...,-19.203181,153.975663,15.39681,0.453202,NaN,NaN,NaN,15.396811,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6234,366,2016,659011,新星市,4.952979,8.761851,10.716098,7335,5933,650000,...,NaN,NaN,15.126213,0.326896,NaN,NaN,NaN,15.126213,NaN,NaN
6235,366,2017,659011,新星市,4.999036,8.860108,10.791734,7336,6304,650000,...,NaN,NaN,15.128802,0.318632,NaN,NaN,NaN,15.128803,NaN,NaN
6236,366,2018,659011,新星市,5.044239,8.955623,10.865422,7337,6675,650000,...,NaN,NaN,15.132703,0.321611,NaN,NaN,NaN,15.132704,NaN,NaN
6237,366,2019,659011,新星市,5.089039,9.049568,10.938036,7338,7046,650000,...,NaN,NaN,15.136254,0.324218,NaN,NaN,NaN,15.136254,NaN,NaN


In [ ]:
merged_df.to_csv("/content/merged_df.csv", encoding = "utf-8")